In [0]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

from sklearn.preprocessing import MultiLabelBinarizer

In [20]:
!wget 'https://storage.googleapis.com/movies_data/movies_metadata.csv'

--2019-07-13 17:20:34--  https://storage.googleapis.com/movies_data/movies_metadata.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.166.128, 2a00:1450:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.166.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34445126 (33M) [application/octet-stream]
Saving to: ‘movies_metadata.csv.1’

movies_metadata.csv 100%[===================>]  32.85M  26.0MB/s    in 1.3s    

2019-07-13 17:20:36 (26.0 MB/s) - ‘movies_metadata.csv.1’ saved [34445126/34445126]



In [21]:
data = pd.read_csv("movies_metadata.csv")

descriptions = data['overview']
genres = data['genres']

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
top_genres = ['Comedy', 'Thriller', 'Romance', 'Action', 'Horror', 'Crime', 'Documentary', 'Adventure', 'Science Fiction']

In [0]:
train_size = int(len(descriptions) * 0.8)

train_descriptions = descriptions[:train_size]
train_genres = genres[:train_size]

test_descriptions = descriptions[train_size:]
test_genres = genres[train_size:]

In [0]:
description_embeddings = hub.text_embedding_column(
    'descriptions',
    module_spec='https://tfhub.dev/google/universal-sentence-encoder/2'
)

In [0]:
encoder = MultiLabelBinarizer()
encoder.fit_transform(train_genres)
train_encoded = encoder.transform(train_genres)
test_encoded = encoder.transform(test_genres)
num_classes = len(encoder.classes_)

In [0]:
multi_label_head = tf.contrib.estimator.multi_label_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

In [27]:
estimator = tf.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64, 10],
    feature_columns=[description_embeddings]
)

W0713 17:20:39.053831 139719292798848 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpbj1jju6c


In [0]:
features = {
    'descriptions': np.array(train_descriptions)
}

labels = np.array(train_encoded)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    features,
    labels,
    shuffle=True,
    batch_size=32,
    num_epochs=20
)

In [29]:
estimator.train(input_fn=train_input_fn)

InternalError: ignored